# Μικρή εισαγωγή στην ανάλυση κειμένου!

## Θα ξεκινήσουμε να αναλύουμε κείμενα και θα χρησιμοποιήσουμε τις παρακάτω βιβλιοθήκες, για τις οποίες μπορείτε να μάθετε περισσότερα αν επισκεφθείτε τις ιστοσελίδες τους.




## TextBlob: "Simplified text processing"

https://textblob.readthedocs.io/en/dev/

## spaCy: "Industrial-Strength Natural Language Processing"

https://spacy.io/

## scikit-learn: "Machine Learning in Python"

http://scikit-learn.org/

## Natural Language Toolkit (NLTK): "A leading platform for building Python programs to work with human language data"

http://www.nltk.org/

# Ας πειραματιστούμε με την ανάλυση συναισθήματος

In [12]:
import nltk #για Επεξεργασία φυσικής γλώσσας Natural Language Toolkit
import pandas #DataFrames
import matplotlib.pyplot as plt #Για γραφικά
import wordcloud #Για word clouds
import numpy as np 
import scipy #Για διαφορές και αποστάσεις
import seaborn as sns #κάνει τα γραφήματα (plot) πιο όμορφα
import sklearn.manifold #Για manifold plot
from nltk.corpus import stopwords #Για stopwords
import json #Για API responses
import urllib.parse #Για ένωση urls

#Για να εμφανίζονται τα γραφήματα
%matplotlib inline


In [1]:
from textblob import TextBlob

blob = TextBlob("I love driving the car.")
blob.sentiment

Sentiment(polarity=0.5, subjectivity=0.6)

In [7]:
tweet = '''I am very disappointed in China. Our foolish past leaders 
have allowed them to make hundreds of billions of dollars a year in trade, yet
they do NOTHING for us with North Korea, just talk. We will no longer allow this
to continue. China could easily solve this problem!'''

TextBlob(tweet).sentiment

Sentiment(polarity=-0.22777777777777777, subjectivity=0.6861111111111112)

Μπορείτε να διαβάσετε το [documentation](https://textblob.readthedocs.io/en/dev/classifiers.html) για περισσότερες πληροφορίες, αλλά εμείς θα μάθουμε περισσότερα για την ανάλυση συναισθήματος παρακάτω.

# Μέτρημα λέξεων με το `.count`


Για να ξεκινήσουμε την ανάλυση πρώτα μετράμε πόσες φορές εμφανίζεται κάθε λέξη στο κείμενο.

In [9]:
"I tried to fish for fish but I didn't catch any fish".count("fish")

3

In [12]:
fish_sentence = "I tried to fish for fish but I didn't catch any fish"
fish_sentence.split(" ")

['I',
 'tried',
 'to',
 'fish',
 'for',
 'fish',
 'but',
 'I',
 "didn't",
 'catch',
 'any',
 'fish']

In [16]:
fish_sentence = "I tried to fish for fish but I didn't catch any fish"
fish_list = fish_sentence.split(" ")
fish_list.count("fish")

3

In [9]:
toucan_sentence = "The canny toucan can't recant about the pelican's scant canteloupe"
toucan_list = toucan_sentence.split(" ")
toucan_list.count("can")

0

Tο μέτρημα των λέξεων είναι αρκετά απλό και μπορεί να γίνει σε παραγράφους και ολόκληρα βιβλία. 

Το πρόβλημα που υπάρχει είναι ότι για παράδειγμα η λέξη can δεν θα μετρηθεί μόνο μία φορά στο παραπάνω παράδειγμα, αλλά όσες φορές εμφανίζεται ακόμα και μέσα σε λέξεις.


In [10]:
"The canny toucan can't recant about the pelican's scant canteloupe".count("can")

7

Η πρόταση έχει `"can"` περισσότερες από μία φορές, οπότε πρέπει να λύσουε αυτό το πρόβλημα και να δοκιμάσουμε κάτι πιο αποτελεσματικό.

# Τokenization
# Φιλτράρισμα και ομαλοποίηση  του κειμένου

Για να αρχίσουμε να συγκρίνουμε τα διαφορετικά κείμενα πρέπει πρώτα να ομαλοποιήσουμε το κείμενο και να φιλτράρουμε αυτά στα οποία στην συνέχεια θα επικεντρωθούμε. Αρχικά, θα κάνουμε όλα τα κεφαλαία, μικρά και θα πετάξουμε τα tokens που δεν είναι λέξεις. Στην συνέχεια, θα αφαιρέσουμε τα 'stop words', θα κάνουμε stem τις λέξεις που θα μείνουν για να φύγουν οι καταλήξεις των λέξεων που δεν μας χρειάζονται (suffixes), ή θα κάνουμε λημματισμό (lemmatize) τα tokens μέσω ένος έξυπνου αλγορίθμου που θα αντιλαμβάνεται τις διαφορετικές μορφές μιας ίδιας λέξης (π.χ. χρησιμοποιώντας έναν stemmer και ένα λεξικό). 



In [18]:
fish_sentence = "I tried to fish for fish but I didn't catch any fish"

In [19]:
fish_sentence = "I tried to fish for fish but I didn't catch any fish"

In [20]:
toucan_sentence = "The canny toucan can't recant the pelican's scant canteloupe"

## Βελτίωση του tokenization

Μπορούμε να καθαρίζουμε τα κενά και να μετράμε λέξεις, αλλά με τα κεφαλαία και τα σημεία στίξης υπάρχουν αρκετά προβλήματα. 

In [14]:
dinner_sentence = "Dinner was great tonight, I enjoyed the potatoes."
dinner_list = dinner_sentence.split(" ")
dinner_list

['Dinner', 'was', 'great', 'tonight,', 'I', 'enjoyed', 'the', 'potatoes.']

In [16]:
# Πόσες φορές εμφανίστηκε η λέξη 'dinner'?
dinner_list.count("dinner")

0

In [17]:
# Πόσες φορές εμφανίστηκε η λέξη 'potatoes'?
dinner_list.count("potatoes")

0

In [24]:
dinner_sentence = "Dinner was great tonight, I enjoyed the potatoes."
dinner_sentence = dinner_sentence.lower().replace(".", " ")
dinner_list = dinner_sentence.split(" ")
dinner_list

['dinner', 'was', 'great', 'tonight,', 'i', 'enjoyed', 'the', 'potatoes', '']

In [25]:
dinner_list.count("dinner")

1

In [26]:
dinner_list.count("potatoes")

1

# Tokenizing με το TextBlob

Το TextBlob είναι εύχρηστο, αλλά όχι τόσο πλήρες όσο το spaCy.

In [2]:
from textblob import TextBlob

In [3]:

blob = TextBlob("The dangerous cats ran dangerously toward dangers.")

for token in blob.tokens:
    print(token)

The
dangerous
cats
ran
dangerously
toward
dangers
.


In [29]:
for word in blob.words:
    print(word)

The
dangerous
cats
ran
dangerously
toward
dangers


Βλέπετε ότι διατηρεί τις φράσεις όταν χρησιμοποιείτε το`.tokens`?

# Tokenizing με το spaCy

Το spaCy μπορεί να κάνει τα πάντα, αλλά είναι πιο δύσκολο από το TextBlob.

In [21]:
import spacy
nlp = spacy.load('en')

In [23]:
doc = nlp("The dangerous cats ran dangerously toward dangers")
tokens = [token for token in doc]
tokens

[The, dangerous, cats, ran, dangerously, toward, dangers]

# Λίγο ακόμη TextBlob


In [36]:
text = "Today I went driving to the grocery store. I hate to drive the car, but I love visiting the gas station!"

## Όλες οι λέξεις

Αυτό είναι σαν το tokens, αλλά αφαιρεί τα σημεία στίξης.

In [37]:
doc = TextBlob(text)
doc.words

WordList(['Today', 'I', 'went', 'driving', 'to', 'the', 'grocery', 'store', 'I', 'hate', 'to', 'drive', 'the', 'car', 'but', 'I', 'love', 'visiting', 'the', 'gas', 'station'])

## Φράσεις με ουσιαστικά

In [38]:
doc = TextBlob(text)
doc.noun_phrases

WordList(['grocery store', 'gas station'])

## Όλες οι προτάσεις

In [39]:
doc = TextBlob(text)
doc.sentences

[Sentence("Today I went driving to the grocery store."),
 Sentence("I hate to drive the car, but I love visiting the gas station!")]

In [41]:
[len(sent.words) for sent in doc.sentences]

[8, 13]

## n-grams

In [42]:
doc = TextBlob("I went to the pet store to buy a fish.")
doc.ngrams(2)

[WordList(['I', 'went']),
 WordList(['went', 'to']),
 WordList(['to', 'the']),
 WordList(['the', 'pet']),
 WordList(['pet', 'store']),
 WordList(['store', 'to']),
 WordList(['to', 'buy']),
 WordList(['buy', 'a']),
 WordList(['a', 'fish'])]

## Λίστα με λέξεις

In [44]:
doc = TextBlob("The dangerous cats ran dangerously toward dangers.")
words = doc.words
words

WordList(['The', 'dangerous', 'cats', 'ran', 'dangerously', 'toward', 'dangers'])

In [45]:
words[2]

'cats'

In [46]:
words[2].pluralize()

'catss'

In [47]:
words[2].pluralize().pluralize().pluralize().pluralize()

'catssesses'

## Στελέχωση (stemming) και Λημματισμός (lemmatization) με TextBlob



In [88]:
blob = TextBlob("The dangerous cats ran dangerously toward dangers.")
words = blob.words

for word in words:
    print("ORIGINAL: ", word)

ORIGINAL:  The
ORIGINAL:  dangerous
ORIGINAL:  cats
ORIGINAL:  ran
ORIGINAL:  dangerously
ORIGINAL:  toward
ORIGINAL:  dangers


In [91]:
blob = TextBlob("The dangerous cats ran dangerously toward dangers.")
words = blob.words

for word in words:
    print("ORIGINAL: ", word, "| LEMMA: ", word.lemmatize(),"STEM: ", word.stem())

ORIGINAL:  The | LEMMA:  The STEM:  the
ORIGINAL:  dangerous | LEMMA:  dangerous STEM:  danger
ORIGINAL:  cats | LEMMA:  cat STEM:  cat
ORIGINAL:  ran | LEMMA:  ran STEM:  ran
ORIGINAL:  dangerously | LEMMA:  dangerously STEM:  danger
ORIGINAL:  toward | LEMMA:  toward STEM:  toward
ORIGINAL:  dangers | LEMMA:  danger STEM:  danger


In [56]:
blob.tokens

WordList(['The', 'dangerous', 'cats', 'ran', 'dangerously', 'toward', 'dangers', '.'])

In [57]:
blob.words

WordList(['The', 'dangerous', 'cats', 'ran', 'dangerously', 'toward', 'dangers'])

Ο λημματισμός στο TextBlob **δεν λειτουργεί καλά** Θεωρεί ότι όλα είναι ουσιαστικά, εκτός και αν του πούμε εμείς ότι δεν είναι. 

In [48]:
TextBlob("running").words[0].lemmatize()

'running'

In [49]:
TextBlob("running").words[0].lemmatize('v')

'run'

# Tokenizing με το  spaCy


In [24]:
import spacy
nlp = spacy.load('en')

In [25]:

doc = nlp("The dangerous cats ran dangerously toward dangers.")
tokens = [token.lemma_ for token in doc]
tokens

['the', 'dangerous', 'cat', 'run', 'dangerously', 'toward', 'danger', '.']

## Το spaCy δεν διαθέτει έτοιμο τρόπο για ανάλυση συναισθήματος, το TextBlob όμως έχει.

In [29]:
phrases = ['i love cars', 'i hate cars', 'i butter cars', 'misery and gloomy pain cars']
for phrase in phrases:
    doc = nlp(phrase)
    print("The sentiment for", doc, "is", doc[0].sentiment)

The sentiment for i love cars is 0.0
The sentiment for i hate cars is 0.0
The sentiment for i butter cars is 0.0
The sentiment for misery and gloomy pain cars is 0.0


In [30]:
words = ['love', 'hate', 'butter', 'misery and gloomy pain']
for word in words:
    blob = TextBlob(word)
    print("The sentiment for", word, "is", blob.sentiment)

The sentiment for love is Sentiment(polarity=0.5, subjectivity=0.6)
The sentiment for hate is Sentiment(polarity=-0.8, subjectivity=0.9)
The sentiment for butter is Sentiment(polarity=0.0, subjectivity=0.0)
The sentiment for misery and gloomy pain is Sentiment(polarity=0.0, subjectivity=0.0)


# Intro to scikit-learn(sklearn)

In [30]:
phrases = ['i love cars', 
           'i hate cars', 
           'cars butter cars', 
           'misery and gloomy pain cars',
           'the cars hate butter'
          ]

# # VECTORIZATION

In [31]:
from sklearn.feature_extraction.text import CountVectorizer


In [32]:
#Δώσε μπου κάτι που να μετρά λέξεις για μένα
vec = CountVectorizer()
#Έχω κάποιες προτάσεις που χρειάζονται μέτρημα
matrix = vec.fit_transform(phrases)
#τι βρήκες??
matrix

<5x9 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [33]:
#Το κείμενό μας έχει γίνει... VECTORIZED!!
matrix.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 2, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [80]:
import pandas as pd
pd.DataFrame(matrix.toarray())

,0,1,2,3,4,5,6,7,8
0,0,0,1,0,0,1,0,0,0
1,0,0,1,0,1,0,0,0,0
2,0,1,2,0,0,0,0,0,0
3,1,0,1,1,0,0,1,1,0
4,0,1,1,0,1,0,0,0,1


In [82]:
vec.get_feature_names()

['and', 'butter', 'cars', 'gloomy', 'hate', 'love', 'misery', 'pain', 'the']

In [86]:
docs = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())

In [87]:
docs.sort_values(by='cars', ascending=False)

,and,butter,cars,gloomy,hate,love,misery,pain,the
2,0,1,2,0,0,0,0,0,0
0,0,0,1,0,0,1,0,0,0
1,0,0,1,0,1,0,0,0,0
3,1,0,1,1,0,0,1,1,0
4,0,1,1,0,1,0,0,0,1
